# PART ONE

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

Import of scientific libraries

- Pandas is used for handling tabular data
- Numpy for numerical comutation
- Matplotlib for visualisation 
- SciPy for statistical analysis


In [4]:
df = pd.read_excel("data.xlsx")
df.head()

,Série,5,5.1,Unnamed: 3,Unnamed: 4,S5
0,RSD:,7.000,4.000,NaN,NaN,C
1,REF,49.130,49.770,NaN,10.0,9.800777
2,1,55.101,49.138,NaN,10.0,12.254704
3,2,44.762,50.282,NaN,10.0,11.590869
4,3,51.454,52.862,NaN,10.0,10.692108


Data loading and inspection

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Série       25 non-null     object 
 1   5           25 non-null     float64
 2   5.1         25 non-null     float64
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  42 non-null     float64
 5   S5          43 non-null     object 
dtypes: float64(4), object(2)
memory usage: 2.1+ KB


In [7]:
df.columns

Index(['Série', 5, '5.1', 'Unnamed: 3', 'Unnamed: 4', 'S5'], dtype='object')

The excel file does not contain a single clean table. Several colomns are empty or contain labels. The dataset was therefore insoected to identify which columns correspond to the comparison test and which correspond to the accuracy test. 

In [9]:
df_raw = df.copy()
df_raw.head()

,Série,5,5.1,Unnamed: 3,Unnamed: 4,S5
0,RSD:,7.000,4.000,NaN,NaN,C
1,REF,49.130,49.770,NaN,10.0,9.800777
2,1,55.101,49.138,NaN,10.0,12.254704
3,2,44.762,50.282,NaN,10.0,11.590869
4,3,51.454,52.862,NaN,10.0,10.692108


Preserve original import

In [11]:
df_raw.iloc[:10, :6]

,Série,5,5.1,Unnamed: 3,Unnamed: 4,S5
0,RSD:,7.000,4.000,NaN,NaN,C
1,REF,49.130,49.770,NaN,10.0,9.800777
2,1,55.101,49.138,NaN,10.0,12.254704
3,2,44.762,50.282,NaN,10.0,11.590869
4,3,51.454,52.862,NaN,10.0,10.692108
5,4,49.648,47.998,NaN,10.0,9.318845
6,5,47.177,47.922,NaN,10.0,10.218197
7,6,53.897,53.682,NaN,20.0,21.63061
8,7,45.294,52.974,NaN,20.0,20.170285
9,8,44.461,49.698,NaN,20.0,20.803307


Identify columns by position

In [13]:
step1 = df_raw.iloc[2:25, 1:3]
step1.columns = ["department_1", "department_2"]
step1 = step1.astype(float)

Data structuring

In [15]:
step1.head()
step1.describe()

,department_1,department_2
count,23.000000,23.000000
mean,49.350043,49.967043
std,4.521143,2.247061
min,42.354000,45.922000
25%,45.444500,48.020000
50%,49.648000,49.698000
75%,53.361500,52.024000
max,55.948000,53.682000


Sanity check

In [17]:
ref_department_1 = df_raw.iloc[1,1]
ref_department_2 = df_raw.iloc[1,2]

ref_department_1, ref_department_2

(49.13, 49.77)

Extract reference values

In [19]:
mean_dep1 = step1["department_1"].mean()
mean_dep2 = step1["department_2"].mean()

mean_dep1,mean_dep2

(49.35004347826088, 49.96704347826088)

The mean concentrations measured by the two departments are approximately 49.35 µg/L and 49.97 µg/L


In [21]:
diff_dep1 = mean_dep1 - ref_department_1
diff_dep2 = mean_dep2 - ref_department_2

diff_dep1, diff_dep2

(0.22004347826087667, 0.19704347826087343)

Quantifying the deviation from the reference

In [23]:
step1_std = step1.std()
step1_std

department_1    4.521143
department_2    2.247061
dtype: float64

Agreement with the reference value:

Differences between mean and reference are:
≈ 0.2 µg/L for both departments

Standard deviations are:
≈ 4.5 µg/L and ≈ 2.3 µg/L

Because:
0.2 ≪ 2–4

the deviation from the reference is negligible compared to the measurement spread.

Homogeneity between laboratories

The mean concentrations measured by the two departments are very close to each other and to the reference values. 

The dispersion of the repeated measurements, as indicated by the standard deviations, shows that department 2 has lower variability and therefore better precision. However, the results produced by both departments are comparable in magnitude.

The two laboratories can therefore be considered homogeneous with respect to the analytical results.

In [26]:
alpha = 0.05
n = len(step1)

def t0_vs_reference(x, mu0):
    xbar = x.mean()
    s = x.std(ddof=1)
    t0 = abs(xbar - mu0) / (s / np.sqrt(len(x)))
    return xbar, s, t0

xbar1, s1, t0_1 = t0_vs_reference(step1["department_1"], ref_department_1)
xbar2, s2, t0_2 = t0_vs_reference(step1["department_2"], ref_department_2)

tcrit = stats.t.ppf(1 - alpha/2, df=n-1)

xbar1, s1, t0_1, xbar2, s2, t0_2, tcrit
    

(49.35004347826088,
 4.521143303496472,
 0.2334125195117724,
 49.96704347826088,
 2.247061343141726,
 0.42054362574074117,
 2.0738730679040147)

On average, departmenrt 1 measured a concentration of about 49.35 µg/L. Individual measrements from department 1 vary around the mean with a typical spread of about 4.52 µg/L. This reflects analytical precision. The uncertainty on the estaimated mean is only about 0.24 µg/L, much smaller than the raw measurement variability. 

ON average, departmnet 2 measured a concentration of about 49.97 µg/L.. Department 2 shows lower dispersion, with a typcial spread of about 2.25 µg/L, indicating better analytical precision than department 1. The uncertainty on the estimated mean is 0.43 µg/L. This is how precisely each lab's mean is known. 

At the 95 % confidence level and for the given sample size, the critical student t value is approx. 2.07. Values below 2.07 are compatible with random variation. Values above 2.07 indicate a statistically significant difference. Since the deviaitons observed in the estimated means remain very much below this critical value, the mean concentration values obtained by the two departments are statistically consistent and in good agreement with the reference values. 


Null hypothesis H0
The laboratory variance equals the reference variance.

s² = σ₀²

Alternative hypothesis H1
The laboratory variance is different from the reference variance.

s² ≠ σ₀²

In [29]:
rsd_dep1 = 7.0
rsd_dep2 = 4.0

In [30]:
# reference standard deviations
sigma0_1 = ref_department_1 * rsd_dep1 / 100
sigma0_2 = ref_department_2 * rsd_dep2 / 100

# sample variances
s2_1 = s1**2
s2_2 = s2**2

# chi-square statistics
chi2_1 = (n - 1) * s2_1 / (sigma0_1**2)
chi2_2 = (n - 1) * s2_2 / (sigma0_2**2)

# chi-square critical value (upper, 95%)
chi2_crit = stats.chi2.ppf(1 - alpha, df=n-1)

chi2_1, chi2_2, chi2_crit

(38.02153254196358, 28.028333326198283, 33.92443847144381)

In [31]:
# reference standard deviations from the assignment table
sigma_ref_1 = 7.0
sigma_ref_2 = 4.0

In [32]:
var_ref_1 = sigma_ref_1**2
var_ref_2 = sigma_ref_2**2

var_ref_1, var_ref_2

(49.0, 16.0)

In [33]:
s1 = step1["department_1"].std(ddof=1)
s2 = step1["department_2"].std(ddof=1)

var1 = s1**2
var2 = s2**2

s1, s2, var1, var2

(4.521143303496472, 2.247061343141726, 20.44073677075099, 5.049284679841898)

In [34]:
n = len(step1)
df = n - 1

chi2_1 = df * var1 / var_ref_1
chi2_2 = df * var2 / var_ref_2

chi2_1, chi2_2

(9.177473652173914, 6.942766434782611)

The conformity of variances with respect to the reference standard deviations was evaluated using a chi-square test. For both department 1 (χ² = 9.18) and department 2 (χ² = 6.94), the calculated values were lower than the critical value at the 95% confidence level (χ²_crit = 33.92), indicating that the variances are consistent with the reference values.

H0: μ1 = μ2m
H1: μ1 ≠ μ2

In [ ]:
n1, n2 = len(x1), len(x2)

xbar1, xbar2 = x1.mean(), x2.mean()
s1, s2 = x1.std(ddof=1), x2.std(ddof=1)

t0 = abs(xbar1 - xbar2) / np.sqrt(s1**2/n1 + s2**2/n2)

tcrit_table = stats.t.ppf(1 - 0.05/2, df=40)

xbar1, s1, xbar2, s2, t0, tcrit_table


This test compares the difference between the two means to the natural variability of the measurements.
- there is no statistically significant difference between the two department means
- the means are homogeneous

The difference between the mean values obtained by the two departments is small relative to the measurement variability. The calculated t statistic (t0 = 0.586) is lower than the critical value at the 95% confidence level, indicating that the two means are statistically homogeneous.

In [ ]:
import numpy as np
from scipy import stats

# x1 and x2 already cleaned and NaNs removed
s1_2 = x1.var(ddof=1)
s2_2 = x2.var(ddof=1)

# ensure larger variance on top
if s1_2 >= s2_2:
    F0 = s1_2 / s2_2
    df1, df2 = len(x1) - 1, len(x2) - 1
else:
    F0 = s2_2 / s1_2
    df1, df2 = len(x2) - 1, len(x1) - 1

# critical value at 95% confidence
F_crit = stats.f.ppf(1 - 0.05, df1, df2)

F0, F_crit


if F0 > Fcrit → variances are significantly different

# PART DEUX

In [ ]:
step2 = df_raw.iloc[2:, 4:6]
step2.columns = ["reference", "measured"]
step2 = step2.dropna()
step2 = step2.astype(float)

step2.head()
step2.tail()

Data structuring

The reference and measured concentration data for the accuracy test were extracted and cleaned to remove non-numerical entries prior to regression analysis.

In [ ]:
plt.figure()
plt.scatter(step2["reference"], step2["measured"])
plt.plot(step2["reference"], step2["reference"])
plt.xlabel("Reference concentration (µg/L)")
plt.ylabel("Measured concentration (µg/L)")
plt.show()


Visualising the accuracy test

The measured concentrations increase proportionally with the reference concentrations, indicating a linear relationship over the application range

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(
    step2["reference"], step2["measured"]
)

slope, intercept

# Accuracy test

- Measured concentrations increase proportionally with the reference concentrations.
- The slope is close to the expected value of 1.
- The intercept is small and close to 0.
- No systematic deviation is observed across the concentration range.
- The method is accurate over the application range.

In [ ]:
r_value**2

$R^2$ close to 1 shows that the standards and measurements follow each other very closely across the range